In [ ]:
# Cell 1: 強制更新套件 (加上 -U)
!pip install -U google-generativeai streamlit pillow

In [ ]:
# API Key 診斷程式
import google.generativeai as genai
import getpass

print("請貼上你的 Gemini API Key 按下 Enter:")
my_key = getpass.getpass()

try:
    genai.configure(api_key=my_key)
    print("\n正在嘗試連線 Google 伺服器...")

    # 1. 測試列出所有可用模型
    print("📋 你的帳號可用的模型清單：")
    count = 0
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f" - {m.name}")
            count += 1

    if count > 0:
        print(f"\n API Key 是有效的，且抓到了 {count} 個模型。")
        print("執行 app.py 的步驟了 (記得 API Key 要貼剛剛這組)。")
    else:
        print("\n API Key 沒有權限存取任何模型。")

except Exception as e:
    print("\n 測試失敗！你的 API Key 有問題，或是套件版本太舊。")
    print(f"完整錯誤訊息: {e}")

請貼上你的 Gemini API Key 按下 Enter:
··········

正在嘗試連線 Google 伺服器...
📋 你的帳號可用的模型清單：
 - models/gemini-2.5-flash
 - models/gemini-2.5-pro
 - models/gemini-2.0-flash-exp
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-exp-image-generation
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-2.0-flash-lite-preview-02-05
 - models/gemini-2.0-flash-lite-preview
 - models/gemini-exp-1206
 - models/gemini-2.5-flash-preview-tts
 - models/gemini-2.5-pro-preview-tts
 - models/gemma-3-1b-it
 - models/gemma-3-4b-it
 - models/gemma-3-12b-it
 - models/gemma-3-27b-it
 - models/gemma-3n-e4b-it
 - models/gemma-3n-e2b-it
 - models/gemini-flash-latest
 - models/gemini-flash-lite-latest
 - models/gemini-pro-latest
 - models/gemini-2.5-flash-lite
 - models/gemini-2.5-flash-image-preview
 - models/gemini-2.5-flash-image
 - models/gemini-2.5-flash-preview-09-2025
 - models/gemini-2.5-flash-lite-preview-09-2025
 - models/gemini-3-pro-preview
 - mo

In [ ]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import json
import time
import urllib.parse
import requests
from io import BytesIO
from PIL import Image
import random

# --- 1. 頁面設定 ---
st.set_page_config(
    page_title="FeastVision (Elite Access)",
    page_icon="✨",
    layout="wide"
)

# --- 2. 核心邏輯：Gemini 識圖與分析 ---
def analyze_menu_gemini(api_key, image):
    genai.configure(api_key=api_key)

    candidate_models = [
        "models/gemini-2.5-flash"
    ]

    prompt = """
    你是一位專業的美食評論家。請分析這張菜單圖片。
    識別每一道菜，並輸出嚴格的 JSON 格式。

    JSON 結構必須包含 "items" 列表，每個物件包含：
    - "original_name": 原文菜名
    - "price": 價格
    - "translated_name": 繁體中文翻譯
    - "description": 簡短介紹
    - "image_prompt": 英文繪圖提示詞。

    【重要指令】
    請為 "Nano Banana" (Stable Diffusion model) 撰寫最佳提示詞。
    風格要求：'Nano Banana style', '2.5D', 'vibrant colors', 'soft lighting', 'highly detailed', 'semi-realistic'.

    直接輸出 JSON string。
    """

    for model_name in candidate_models:
        try:
            print(f"嘗試模型: {model_name}...")
            model = genai.GenerativeModel(model_name)
            response = model.generate_content([prompt, image])
            text = response.text.replace("```json", "").replace("```", "").strip()
            parsed_json = json.loads(text)
            st.toast(f"✅ 使用模型: {model_name}")
            return parsed_json
        except Exception as e:
            print(f"模型 {model_name} 失敗: {e}")
            continue

    st.error("所有模型嘗試失敗，請檢查 API Key。")
    return None

# --- 3. 核心邏輯：生圖 ---
def generate_image_url(prompt):
    clean_prompt = str(prompt)[:250]
    # 強制注入風格
    style_suffix = ", (Nano Banana style:1.2), 2.5D, vibrant colors, soft lighting, anime aesthetic"
    final_prompt = clean_prompt + style_suffix

    encoded_prompt = urllib.parse.quote(final_prompt)
    seed = random.randint(0, 999999)
    # 使用 Turbo 確保速度
    url = f"https://image.pollinations.ai/prompt/{encoded_prompt}?model=turbo&width=1024&height=1024&nologo=true&seed={seed}"
    return url

# --- 安全下載圖片函式 ---
def download_image(url):
    for attempt in range(2): # 試 2 次就好，失敗就切換方案 B
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return Image.open(BytesIO(response.content))
        except Exception:
            time.sleep(1)
    return None

# --- 4. Session State ---
if "menu_data" not in st.session_state:
    st.session_state.menu_data = None
if "generated_images" not in st.session_state:
    st.session_state.generated_images = {}

# --- 5. 前端介面 ---
st.title("✨ 異國菜單具象化")
st.caption("Accessing: Gemini 2.5 / 3.0 / Nano Banana Preview")

with st.sidebar:
    st.header("⚙️ 設定")
    api_key = st.text_input("請輸入 API Key", type="password", key="api_key_final")

    if st.button("清除重來"):
        st.session_state.menu_data = None
        st.session_state.generated_images = {}
        st.rerun()

uploaded_file = st.file_uploader("📸 上傳菜單 (JPG/PNG)", type=["jpg", "jpeg", "png"])

if uploaded_file and api_key:
    image = Image.open(uploaded_file)
    st.image(image, caption="原始菜單", use_container_width=True)

    if st.button("🚀 開始分析"):
        with st.spinner("正在分析中..."):
            result = analyze_menu_gemini(api_key, image)
            if result:
                st.session_state.menu_data = result.get("items", [])
                st.success(f"成功辨識 {len(st.session_state.menu_data)} 道菜！")
                st.rerun()

if st.session_state.menu_data:
    st.divider()
    st.subheader("📋 分析結果")
    for idx, item in enumerate(st.session_state.menu_data):
        with st.container():
            col1, col2 = st.columns([3, 2])
            with col1:
                st.markdown(f"### {idx+1}. {item['translated_name']} ({item['price']})")
                st.markdown(f"**原文：** `{item['original_name']}`")
                st.markdown(f"**介紹：** {item['description']}")
            with col2:
                if idx in st.session_state.generated_images:
                    content = st.session_state.generated_images[idx]

                    # 方案 A: 圖片物件 (伺服器下載成功)
                    if isinstance(content, Image.Image):
                        st.image(content, caption="AI 預覽圖", use_container_width=True)

                    # 方案 B: 網址字串 (伺服器下載失敗，改用瀏覽器顯示)
                    elif isinstance(content, str):
                        # 使用 Markdown 語法顯示圖片，這會由你的瀏覽器去抓圖，絕對不會被擋
                        st.markdown(f"![AI 預覽圖]({content})")
                        st.caption("AI 預覽圖 (瀏覽器載入)")
                else:
                    if st.button(f"✨ 生成圖片", key=f"btn_{idx}"):
                        status_box = st.empty()
                        status_box.info("AI 繪圖中...")

                        img_url = generate_image_url(item['image_prompt'])

                        # 嘗試下載
                        img = download_image(img_url)

                        if img:
                            st.session_state.generated_images[idx] = img
                        else:
                            # 下載失敗沒關係，我們存網址，讓前端 Markdown 自己去抓
                            st.session_state.generated_images[idx] = img_url

                        status_box.empty()
                        st.rerun()
            st.divider()

Writing app.py


In [ ]:
# Cell 3: 啟動 Streamlit 並使用 Cloudflare Tunnel (推薦)
import subprocess
import time

print("正在安裝 Cloudflare Tunnel...")
# 下載並安裝 cloudflared
!curl -L --output cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb > /dev/null 2>&1
!rm cloudflared.deb

print("正在啟動伺服器...")
# 背景執行 Streamlit
!streamlit run app.py > /dev/null 2>&1 &

# 等待 Streamlit 啟動
time.sleep(3)

print("正在建立隧道 (這可能需要 10-15 秒)...")
# 背景執行 Cloudflare Tunnel 並將輸出導向檔案以便讀取網址
!cloudflared tunnel --url http://localhost:8501 > tunnel_log.txt 2>&1 &

# 讀取並顯示網址
time.sleep(5)
print("👇 請點擊下方連結進入你的專題 (無需密碼)：")
!grep -o 'https://.*\.trycloudflare.com' tunnel_log.txt | head -n 1

正在安裝 Cloudflare Tunnel...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 19.2M  100 19.2M    0     0  11.6M      0  0:00:01  0:00:01 --:--:-- 97.8M
正在啟動伺服器...
正在建立隧道 (這可能需要 10-15 秒)...
👇 請點擊下方連結進入你的專題 (無需密碼)：
https://house-expectations-fame-fell.trycloudflare.com
